In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=18c4e4e72084e66f214b5c3924cfa81361ed84306367eec9cca7f467c5a3027b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark").getOrCreate()

In [3]:

weather_data=spark.read.csv("/content/Weather Data in India from 1901 to 2017.csv",header=True)
weather_data.show()

+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|  1|1902|   19|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|  6|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|  7|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|  8|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|  9

**Question 1**

Add column average temperature for each year

In [4]:

from pyspark.sql.functions import *
list=[month for month in weather_data.columns[2:]]
exp='+'.join(list)
weather_data=weather_data.withColumn("Total",expr(exp)/12)
weather_data.show()



+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|             Total|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|
|  1|1902|   19|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|24.099999999999998|
|  6|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|

**Question 2**

Add row average temperature of each month

In [5]:

weather_data2=weather_data.select([months for months in weather_data.columns if months!="YEAR" and months!='_c0'])
avg_dict={}
for i in weather_data2.columns:
  avg_dict[i]=weather_data2.select(i).agg({i:"avg"}).collect()[0][0]
avg_dict

{'JAN': 18.42324786324786,
 'FEB': 20.137948717948717,
 'MAR': 23.434188034188036,
 'APR': 26.514102564102565,
 'MAY': 28.386410256410255,
 'JUN': 28.300940170940162,
 'JUL': 27.369230769230775,
 'AUG': 26.940085470085464,
 'SEP': 26.342649572649577,
 'OCT': 24.742051282051282,
 'NOV': 21.765726495726508,
 'DEC': 19.173333333333332,
 'Total': 24.294159544159545}

In [6]:

from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
new_row = Row(_c0='Average', YEAR='----', JAN=avg_dict['JAN'], FEB=avg_dict['FEB'], MAR=avg_dict['MAR'], APR=avg_dict['APR'],
                                          MAY=avg_dict['MAY'], JUN=avg_dict['JUN'], JUL=avg_dict['JUL'], AUG=avg_dict['AUG'],
                                          SEP=avg_dict['SEP'], OCT=avg_dict['OCT'], NOV=avg_dict['NOV'], DEC=avg_dict['DEC'],
                                          Total=avg_dict['Total']
                                          )

new_df=spark.createDataFrame([new_row])
weather_data=weather_data.union(new_df)
last_df=spark.createDataFrame(weather_data.tail(10))
last_df.show()



+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    _c0|YEAR|              JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|             Total|
+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    108|2009|            19.79|             21.66|             24.55|             27.35|             28.71|             28.77|             27.83|             27.85|             27.11|              25.2|             2

**Question 3**

Add column min and max for each year with corresponding values.


In [7]:
list=[month for month in weather_data.columns[2:-1]]
weather_data=weather_data.withColumn("Maximum",greatest(*list))\
                         .withColumn("Minimum",least(*list))
weather_data.show()


+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+-------+-------+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|             Total|Maximum|Minimum|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+-------+-------+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|   28.6|  17.99|
|  1|1902|   19|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|  28.68|  18.78|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|  28.41|  18.29|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|  27.85|  17.77|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|  28.69|   17.4|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.8

**Question 4**

Create dataframe called 'decade', storing average temperatures of each month in the respective decade. Decade 1910 is from 1901 to 1910.

In [8]:
decade = weather_data.groupBy(floor(col("YEAR") / 10) * 10+10).agg(avg("JAN").alias('JAN'),avg("FEB").alias('FEB'),avg("MAR").alias('MAR'),avg("APR").alias('APR'),avg("MAY").alias('MAY'),
    avg("JUN").alias('JUN'),avg("JUL").alias('JUL'),avg("AUG").alias('AUG'),avg("SEP").alias('SEP'),avg("OCT").alias('OCT'),avg("NOV").alias('NOV'),avg("DEC").alias('DEC'))


decade=decade.withColumnRenamed('floor(col("YEAR") / 10) * 10+10','YEAR')
decade.show()


+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|((floor((YEAR / 10)) * 10) + 10)|               JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|
+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                            1950|            18.144|            19.742|            23.213|              26.4|            28.551|28.359999999999996|27.347999999999995|26.928999999999995|26.264000000000003|24.700

In [9]:
#5.Which year is the hottest year? (Based on avg temp)
hottest_year=weather_data.select("YEAR","Total").orderBy("Total",ascending=False).first()[0]
hottest_year


'2016'

In [10]:
#6.Which year is the coldest year? (Based on avg temp)
coldest_year=weather_data.select("YEAR","Total").orderBy("Total").first()[0]
coldest_year

'1917'

In [11]:
#7.Find out which years recorded minimum and maximum temperatures. (Not based on avg temp)
minimum_temp_year=weather_data.select("YEAR","Minimum").orderBy("Minimum").first()[0]
maximum_temp_year=weather_data.select("YEAR","Maximum").orderBy("Maximum",ascending=False).first()[0]
print("Maximum Year:",maximum_temp_year)
print("Minimum Year:",minimum_temp_year)


Maximum Year: 1921
Minimum Year: 1918


In [14]:
#8.How much raise is observed in min and max temperature for each month?
weather_data4= weather_data.select([cols for cols in weather_data.columns[1:-1]])
weather_data4.show()
for i in weather_data4.columns[:-1]:
  print(f"{i}:{weather_data4.select(i).agg({i,'max'}).collect()[0][0]-weather_data4.select(i+1).agg({i+1,'max'}).collect()[0][0]}")

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+-------+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|             Total|Maximum|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+-------+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|   28.6|
|1902|   19|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|  28.68|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|  28.41|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|  27.85|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|  28.69|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|24.099999999999998|  29.06|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.2

AssertionError: ignored